In [ ]:
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import emcee
from scipy.stats import beta
import matplotlib.pyplot as plt
import numpy as np

#Estimation of the DLM model
## **$ $**
<h2><strong>\( Y_t = F_t \theta_t + X_t \eta + Z_t \zeta + \upsilon_t \)</strong></h2>
<h2><strong>\( \theta_t = G\theta_{t-1} + Z_{t-1} \gamma + \omega_t \)</strong></h2>






## **Estimation procedure using GD**

In [ ]:
# Define the negative log-likelihood function
def negative_log_likelihood(params, Y_t, X_t, Z_t):
    G, *coeffs = params
    eta = np.array(coeffs[:X_t.shape[1]])
    zeta = np.array(coeffs[X_t.shape[1]:])

    T = len(Y_t)
    theta_t = np.zeros(T)
    neg_log_likelihood = 0

    for t in range(T):
        if t > 0:
            theta_t[t] = G * theta_t[t-1] + np.dot(Z_t[t-1], zeta / 2)

        predicted_Y_t = theta_t[t] + np.dot(X_t[t], eta) + np.dot(Z_t[t], zeta / 2)
        neg_log_likelihood += 0.5 * ((Y_t[t] - predicted_Y_t)**2)
    return neg_log_likelihood

# Gradient descent implementation
def estimation_gradient_descent(Y_t, X_t, Z_t, initial_params, learning_rate=0.001, n_iterations=500):
    params = np.array(initial_params)
    n_params = len(params)

    for iteration in range(n_iterations):
        gradients = np.zeros(n_params)

        # Compute gradients
        for i in range(n_params):
            original_param = params[i]

            # Compute loss at original parameter
            loss_original = negative_log_likelihood(params, Y_t, X_t, Z_t)

            # Perturb parameter
            params[i] = original_param + 1e-5
            loss_perturbed = negative_log_likelihood(params, Y_t, X_t, Z_t)

            # Compute gradient
            gradients[i] = (loss_perturbed - loss_original) / 1e-5

            # Reset parameter
            params[i] = original_param

        # Update parameters
        params -= learning_rate * gradients

        if iteration % 10 == 0:
            print(f"Iteration {iteration}: Loss = {loss_original}")
            if np.isnan(loss_original):
                print("NaN encountered in loss. Exiting.")
                break

    return params